In [26]:
import tensorflow as tf
import numpy as np
import random
from collections import deque
import osmnx as ox
import networkx as nx
from matplotlib import cm
import matplotlib.colors as mcolors
from geopy.geocoders import Nominatim
from collections import defaultdict
import time


In [27]:
# Fungsi untuk mengambil koordinat lokasi berdasarkan nama tempat (menggunakan geopy)
def get_location_coordinates(location_name):
    geolocator = Nominatim(user_agent="route_optimizer")
    try:
        location = geolocator.geocode(location_name)
        if location:
            return (location.latitude, location.longitude)
        else:
            print(f"Location {location_name} not found.")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

In [30]:
# Fungsi untuk mengambil input lokasi dan kendaraan
def get_vehicle_and_route_input():
    # Input lokasi awal dan tujuan
    start_location_name = input("Enter start location: ")
    end_location_name = input("Enter destination location: ")

    # Input jenis kendaraan dan status baterai
    vehicle_type = input("Enter vehicle type (car/motorcycle): ").lower()
    if vehicle_type not in ['car', 'motorcycle']:
        print("Invalid vehicle type! Please enter 'car' or 'motorcycle'.")
        return None, None, None, None
    
    # Pengecekan untuk battery level
    try:
        battery_level = float(input("Enter battery level (%): "))
        if battery_level < 0 or battery_level > 100:
            print("Invalid battery level! Please enter a value between 0 and 100.")
            return None, None, None, None
    except ValueError:
        print("Invalid input! Please enter a numeric value for battery level.")
        return None, None, None, None

    # Dapatkan koordinat lokasi
    start_location = get_location_coordinates(start_location_name)
    end_location = get_location_coordinates(end_location_name)

    # Tampilkan hasil input
    print(f"Start Location: {start_location_name}, Coordinates: {start_location}")
    print(f"End Location: {end_location_name}, Coordinates: {end_location}")
    print(f"Vehicle Type: {vehicle_type}")
    print(f"Battery Level: {battery_level}%")

    if not start_location or not end_location:
        print("Error: Invalid locations.")
        return None, None, None, None

    return start_location, end_location, vehicle_type, battery_level


In [51]:
# Mengambil input lokasi dan kendaraan
start_location, end_location, vehicle_type, battery_level = get_vehicle_and_route_input()



Start Location: graha pena surabaya, Coordinates: (-7.3202793, 112.73140395762235)
End Location: tunjungan plaza surabaya, Coordinates: (-7.26238685, 112.73900597862385)
Vehicle Type: car
Battery Level: 90.0%


In [53]:
#code baru
# Data SPKLU yang dimasukkan manual
def get_spklu_locations():
    spklu_locations = [
        (-7.28906, 112.67560),  # EVCuzz Stasiun Pengisian
        (-7.28100, 112.67850),  # CHARGE+ Stasiun Pengisian
        (-7.28110, 112.74630),  # SPKLU Stasiun Pengisian
        (-7.26000, 112.74850),  # Hyundai Stasiun Pengisian
        (-7.25890, 112.74600)   # CHARGE+ Stasiun Pengisian
    ]
    return spklu_locations

# Fungsi untuk memproses rute dan menghitung jarak
def process_route(state, action, spklu_nodes):
    # Misalnya kita dapat menghitung jarak dari start location ke lokasi SPKLU berdasarkan action yang dipilih
    spklu_location = spklu_nodes[action]  # Lokasi SPKLU berdasarkan aksi yang dipilih
    # Misalnya, hitung jarak antara start_location dan spklu_location
    distance = np.linalg.norm(np.array([state[0][0], state[0][1]]) - np.array([spklu_location[0], spklu_location[1]]))
    return distance

# Ambil lokasi SPKLU (hanya koordinat)
spklu_locations = get_spklu_locations()

# Fetch road network with extended distance
def fetch_road_network(depot_location, dist=25000):
    # Fetch road network around the depot location with a radius of dist meters
    return ox.graph_from_point(depot_location, dist=dist, network_type='drive')

# Fetch road network around the start location
G = fetch_road_network(start_location, dist=25000)  # Use the start location as the center point

# Fungsi untuk memetakan lokasi ke node terdekat di jaringan jalan
def map_to_nearest_nodes(locations, graph, distance_threshold=5000):
    valid_nodes = []
    for lat, lng in locations:
        nearest_node = ox.nearest_nodes(graph, lng, lat)
        valid_nodes.append(nearest_node)
    return valid_nodes

# Pemetaan lokasi SPKLU ke node terdekat
spklu_nodes = map_to_nearest_nodes(spklu_locations, G)

print(f"SPKLU Nodes: {spklu_nodes}")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


SPKLU Nodes: [7919104250, 1720870680, 4318089449, 1283196205, 420032173]


In [54]:
#code baru
# DQN setup - Initialize the Q-network, replay memory, and parameters
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # Discount factor
        self.epsilon = 1.0  # Exploration-exploitation tradeoff
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(24, input_dim=self.state_size, activation='relu'),
            tf.keras.layers.Dense(24, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.max(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [56]:
# Inisialisasi DQN agent
state_size = 4  # Contoh, bisa lebih kompleks berdasarkan state yang kamu tentukan
action_size = len(spklu_locations)  # Berdasarkan banyaknya opsi rute yang tersedia
agent = DQNAgent(state_size, action_size)

# Proses untuk mengoptimasi rute
state = np.array([start_location[0], start_location[1], battery_level, 0]).reshape(1, state_size)  # contoh state
action = agent.act(state)

# Output rute yang diambil
print(f"Action taken by DQN Agent: {action}")

print(f"SPKLU chosen: {spklu_nodes[action]}") 

# Proses jarak ke SPKLU berdasarkan action
distance = process_route(state, action, spklu_locations)
print(f"Distance to chosen SPKLU: {distance} km")



Action taken by DQN Agent: 2
SPKLU chosen: 4318089449
Distance to chosen SPKLU: 0.04191550580640705 km


In [12]:
# # Data SPKLU yang dimasukkan manual
# def get_spklu_locations():
#     spklu_locations = [
#         (-7.28906, 112.67560),  # EVCuzz Stasiun Pengisian
#         (-7.28100, 112.67850),  # CHARGE+ Stasiun Pengisian
#         (-7.28110, 112.74630),  # SPKLU Stasiun Pengisian
#         (-7.26000, 112.74850),  # Hyundai Stasiun Pengisian
#         (-7.25890, 112.74600)   # CHARGE+ Stasiun Pengisian
#     ]
#     return spklu_locations

# # Ambil lokasi SPKLU (hanya koordinat)
# spklu_locations = get_spklu_locations()

In [7]:
# Fetch road network with extended distance
def fetch_road_network(depot_location, dist=25000):
    # Fetch road network around the depot location with a radius of dist meters
    return ox.graph_from_point(depot_location, dist=dist, network_type='drive')

# Fetch road network around the start location
G = fetch_road_network(start_location, dist=25000)  # Use the start location as the center point

# Fungsi untuk memetakan lokasi ke node terdekat di jaringan jalan
def map_to_nearest_nodes(locations, graph, distance_threshold=5000):
    valid_nodes = []
    valid_demands = []
    for idx, (lat, lng) in enumerate(locations):
        nearest_node = ox.nearest_nodes(graph, lng, lat)
        nearest_point = (graph.nodes[nearest_node]['y'], graph.nodes[nearest_node]['x'])
        dist_to_nearest = ox.distance.great_circle(lat, lng, nearest_point[0], nearest_point[1])
        print(f"Location {idx}: Distance to nearest node = {dist_to_nearest:.2f} meters")
        if dist_to_nearest > distance_threshold:  # Skip locations too far from the network
            print(f"Warning: Location {idx} is far from the road network! Skipping this location.")
            continue
        valid_nodes.append(nearest_node)
        if idx > 0:  # Exclude depot from demands
            valid_demands.append(10)  # As an example, set demand to 10 for valid nodes
    return valid_nodes, valid_demands

# Pemetaan lokasi ke jaringan jalan
locations = [start_location, end_location] + spklu_locations  # Lokasi awal, tujuan, dan SPKLU
valid_nodes, valid_demands = map_to_nearest_nodes(locations, G)

# Convert graph to undirected for connectivity check
G_undirected = G.to_undirected()

# Periksa konektivitas graph
if not nx.is_connected(G_undirected):
    print("Warning: The road network graph is not fully connected. Consider increasing the distance or changing locations.")
else:
    print("The road network graph is fully connected.")

# Membuat matriks jarak antar node
def build_distance_matrix(valid_nodes, graph):
    distance_matrix = np.zeros((len(valid_nodes), len(valid_nodes)))
    paths = {}
    for i, start_node in enumerate(valid_nodes):
        for j, end_node in enumerate(valid_nodes):
            if i != j:
                try:
                    path = nx.shortest_path(graph, start_node, end_node, weight='length')
                    distance = nx.shortest_path_length(graph, start_node, end_node, weight='length')
                    distance_matrix[i][j] = distance
                    paths[(i, j)] = path
                except nx.NetworkXNoPath:
                    print(f"Warning: No path between nodes {start_node} and {end_node}")
                    distance_matrix[i][j] = float('inf')
                    paths[(i, j)] = []
    return distance_matrix, paths

# Bangun matriks jarak dan jalur
distance_matrix, paths = build_distance_matrix(valid_nodes, G)

# Definisikan konsumsi daya per kilometer untuk kendaraan
def energy_consumption_per_km(vehicle_type):
    # Misalnya, konsumsi daya mobil lebih besar daripada motor
    consumption = {
        'car': 0.2,  # kWh per km untuk mobil
        'motorcycle': 0.1  # kWh per km untuk motor
    }
    return consumption.get(vehicle_type, 0.2)  # Default ke mobil jika tidak ditemukan

# Fungsi untuk mencari SPKLU terdekat sepanjang rute
def find_spklu_along_route(path, spklu_locations, graph):
    spklu_on_route = []
    for spklu in spklu_locations:
        spklu_node = ox.nearest_nodes(graph, spklu[1], spklu[0])
        for i in range(len(path) - 1):
            start_node = path[i]
            end_node = path[i + 1]
            if start_node == spklu_node or end_node == spklu_node:
                spklu_on_route.append(spklu)
                break
    return spklu_on_route

# Menghitung daya yang digunakan sepanjang rute
def calculate_energy_usage(path, graph, vehicle_type):
    total_distance = 0
    for i in range(len(path) - 1):
        start_node = path[i]
        end_node = path[i + 1]
        distance = graph[start_node][end_node].get('length', 0)  # Dapatkan jarak antar node
        total_distance += distance
    
    # Konsumsi daya per kilometer
    consumption_rate = energy_consumption_per_km(vehicle_type)
    energy_used = total_distance * consumption_rate / 1000  # Mengkonversi meter ke kilometer
    return energy_used

# Cek apakah kendaraan perlu mengisi daya dan cari SPKLU yang dilalui
battery_threshold = 20  # Baterai di bawah 20% harus mengisi daya
if battery_level < battery_threshold:
    # Tentukan rute dari titik awal ke tujuan
    start_node = ox.nearest_nodes(G, start_location[1], start_location[0])
    end_node = ox.nearest_nodes(G, end_location[1], end_location[0])
    path_to_end = nx.shortest_path(G, start_node, end_node, weight='length')
    print(f"Route to destination: {path_to_end}")
    
    # Cari SPKLU yang dilalui sepanjang rute
    spklu_on_route = find_spklu_along_route(path_to_end, spklu_locations, G)
    if spklu_on_route:
        print(f"SPKLU along the route: {spklu_on_route}")
    else:
        print("No SPKLU found along the route.")
    
    # Menghitung energi yang digunakan untuk mencapai tujuan
    energy_needed = calculate_energy_usage(path_to_end, G, vehicle_type)
    print(f"Energy needed to reach destination: {energy_needed:.2f} kWh")
    
    # Jika energi yang dibutuhkan melebihi kapasitas baterai, cari SPKLU terdekat untuk mengisi daya
    if energy_needed > battery_level:
        print("Battery is insufficient, finding nearest SPKLU for charging.")
        nearest_spklu = min(spklu_on_route, key=lambda spklu: ox.distance.great_circle(start_location[0], start_location[1], spklu[0], spklu[1]))
        print(f"Nearest SPKLU to route: {nearest_spklu}")
        # Tentukan rute ke SPKLU terdekat dari lokasi saat ini
        spklu_node = ox.nearest_nodes(G, nearest_spklu[1], nearest_spklu[0])
        path_to_spklu = nx.shortest_path(G, start_node, spklu_node, weight='length')
        print(f"Route to nearest SPKLU: {path_to_spklu}")
        energy_to_spklu = calculate_energy_usage(path_to_spklu, G, vehicle_type)
        print(f"Energy needed to reach nearest SPKLU: {energy_to_spklu:.2f} kWh")
else:
    print("Battery is sufficient to complete the journey.")
    

# Print summary
print(f"Valid nodes: {valid_nodes}")
print(f"Distance matrix:\n{distance_matrix}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


Location 0: Distance to nearest node = 131.05 meters
Location 1: Distance to nearest node = 159.02 meters
Location 2: Distance to nearest node = 135.51 meters
Location 3: Distance to nearest node = 66.18 meters
Location 4: Distance to nearest node = 43.98 meters
Location 5: Distance to nearest node = 135.75 meters
Location 6: Distance to nearest node = 32.87 meters
The road network graph is fully connected.
Battery is sufficient to complete the journey.
Valid nodes: [5593639586, 5579915086, 7919104250, 1720870680, 4318089449, 1283196205, 420032173]
Distance matrix:
[[    0.     6974.15  10751.025 10015.155  4869.606  7748.133  8120.79 ]
 [ 8633.339     0.    10453.169  9269.036  4853.203  3076.849  2958.776]
 [11050.621 10292.888     0.     1272.507  9725.6   11442.806 11447.8  ]
 [10396.883  9639.15   2455.221     0.     9071.862 10789.068 10794.062]
 [ 5557.753  2884.082 10326.227  9241.457     0.     3234.819  3607.476]
 [ 7874.156  2790.468 11951.68  10866.91   2860.609     0.     